In [1]:
import feedparser
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd

url = "http://news.google.com/news?q=covid&hl=en-US&sort=date&gl=US&num=10&output=rss"

class ParseFeed():
    def __init__(self, url):
        self.feed_url = url
        
    def clean(self, html):
        '''
        Get the text from html and do some cleaning
        '''
        soup = BeautifulSoup(html)
        text = soup.get_text()
        text = text.replace('\xa0', ' ')
        return text

    def parse(self):
        newsDescription = []
        newsUrl = []
        newsDate = []
        
        '''
        Parse the URL, and print all the details of the news 
        '''        
        feeds = feedparser.parse(self.feed_url).entries
        
        for f in feeds:
            newsDescription.append(self.clean(f.get("description", "")))
            newsUrl.append(f.get("link", ""))
            newsDate.append(f.get("published", ""))
            
        df = pd.DataFrame({'Description':newsDescription, 'URL':newsUrl, 'Date Published':newsDate})            
        return df
    
htmlOutput = ""
feed = ParseFeed(url)
df = feed.parse()

In [2]:
df['Date'] = pd.to_datetime(df['Date Published'])
df = df.head(10).sort_values(by = ['Date'], ascending=False)

# Saving data to html output
htmlOutput = """<html><title>COVID-19 Daily News</title><body><style>
		body {
		  background-image: url('background.jpg');
		  background-repeat: no-repeat;
		  background-attachment: fixed;
		  background-size: cover;
		}
	  </style><center><h1>COVID-19 Daily News</h1></center><dl>"""

for index,row in df.iterrows():
    htmlOutput = htmlOutput + "<dt>" + row['Description'] + "</dt><dd>" + row['Date Published'] + "</dd><dd><a href =" + row['URL'] + ">" + row['URL'] + "</a></dd><br>"
htmlOutput = htmlOutput + "</dl></body></html>"
text_file = open("home.html", "w")
text_file.write(htmlOutput)
text_file.close()

df.to_csv('data.csv', index=False)
df

,Description,URL,Date Published,Date
3,Live Updates: COVID-19 news from across the Un...,https://www.cbsnews.com/live-updates/covid-19-...,"Thu, 29 Oct 2020 18:33:45 GMT",2020-10-29 18:33:45+00:00
7,Gilead’s Covid-19 Drug is Mediocre. It Will Be...,https://www.nytimes.com/2020/10/29/health/covi...,"Thu, 29 Oct 2020 18:13:00 GMT",2020-10-29 18:13:00+00:00
4,U.S. sets Covid-19 record with Election Day lo...,https://www.nbcnews.com/news/us-news/u-s-sets-...,"Thu, 29 Oct 2020 17:57:00 GMT",2020-10-29 17:57:00+00:00
9,Can public education return to normal after th...,https://www.brookings.edu/blog/brown-center-ch...,"Thu, 29 Oct 2020 17:39:22 GMT",2020-10-29 17:39:22+00:00
1,COVID-19 Daily Update 10-29-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Thu, 29 Oct 2020 14:01:32 GMT",2020-10-29 14:01:32+00:00
2,India's COVID-19 cases have declined rapidly—b...,https://www.sciencemag.org/news/2020/10/indias...,"Thu, 29 Oct 2020 12:40:00 GMT",2020-10-29 12:40:00+00:00
5,Latest on COVID-19 in MN: Daily cases shatter ...,https://www.mprnews.org/story/2020/10/29/lates...,"Thu, 29 Oct 2020 10:00:00 GMT",2020-10-29 10:00:00+00:00
0,"Coronavirus (COVID-19) Update: October 28, 202...",https://www.fda.gov/news-events/press-announce...,"Wed, 28 Oct 2020 21:51:00 GMT",2020-10-28 21:51:00+00:00
8,SARS-CoV-2 Neutralizing Antibody LY-CoV555 in ...,https://www.nejm.org/doi/full/10.1056/NEJMoa20...,"Wed, 28 Oct 2020 21:02:21 GMT",2020-10-28 21:02:21+00:00
6,COVID-19 Daily Update 10-28-2020 West Virgini...,https://dhhr.wv.gov/News/2020/Pages/COVID-19-D...,"Wed, 28 Oct 2020 14:05:05 GMT",2020-10-28 14:05:05+00:00
